# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Imports

In [1]:
# Standard libs
import pandas as pd
import numpy as np
import math

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

**change dtype for floors to float (required for calculating the norm later in the assignment)**

In [2]:
sales_dtype={'bathrooms':float, 'waterfront':int, 'sqft_above':int,
             'sqft_living15':float, 'grade':int, 'yr_renovated':int,
             'price':float, 'bedrooms':float, 'zipcode':str, 'long':float,
             'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int,
             'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str,
             'sqft_lot':int, 'view':int}

In [3]:
sales = pd.read_csv("./data/kc_house_data.csv", dtype=sales_dtype)
sales.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


# Import useful functions from previous notebook

Copy and paste get_numpy_data() from the second notebook of Week 2.

In [4]:
def get_numpy_data(data_frame, features, output):
    nb_observations = data_frame.shape[0]
    feature_matrix = np.concatenate((np.ones((nb_observations,1)),
                                     data_frame.as_matrix(columns=features)),axis=1)
    output_array = data_frame.as_matrix(columns=output)
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):   
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, ord=2, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print(np.divide(X,norms)) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, ord=2, axis=0)
    return(np.divide(feature_matrix,norms),norms)

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ["sqft_living", "bedrooms"]
my_output = ["price"]
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

(**change from course notebook: set a column vector**)

In [13]:
weights = np.array([1., 4., 1.]).reshape(-1,1)

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [15]:
def compute_ro_i(feature_matrix, output, weights, feature_i):
    prediction = predict_output(feature_matrix, weights)
    # Matrix version
    ro_i = np.dot(feature_matrix[:,feature_i],
                  (output - prediction + weights[feature_i]*feature_matrix[:,feature_i].reshape(-1,1)))
    # return as scalar
    return(ro_i[0])

In [16]:
ro_0 = compute_ro_i(simple_feature_matrix, output, weights, 0)
ro_1 = compute_ro_i(simple_feature_matrix, output, weights, 1)
ro_2 = compute_ro_i(simple_feature_matrix, output, weights, 2)
print("ro_0", ro_0)
print("ro_1", ro_1)
print("ro_2", ro_2)

ro_0 79400300.0145
ro_1 87939470.8233
ro_2 80966698.6662


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

** Any value > than ro_2 \* 2 but < than ro_1 \* 2**

In [47]:
print("Value for Lambda:",ro_2*2,"> Lambda <", ro_1*2)

Value for Lambda: 161933397.332 > Lambda < 175878941.647


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

** Any value > than ro_1 \* 2**

In [18]:
print("Value for Lambda: Any value >",ro_1*2)

Value for Lambda: Any value > 175878941.647


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [19]:
def lasso_coordinate_descent_step(feature_i, feature_matrix, output, weights, l1_penalty):
    
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    
    # compute ro[i]
    ro_i = compute_ro_i(feature_matrix, output, weights, feature_i)

    if feature_i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

**cell modified to transform ouput and weights in vector column**

In [20]:
# should print 0.425558846691
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]).reshape(-1,1), np.array([1., 4.]).reshape(-1,1), 0.1))

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [21]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    nb_features = initial_weights.shape[0]
    max_weights_change = tolerance 
    weights = initial_weights
    while (max_weights_change >= tolerance):
        max_weights_change = 0
        for feature_i in range (0,nb_features):
            old_weights_i = weights[feature_i][0]
            weights[feature_i] = lasso_coordinate_descent_step(feature_i, feature_matrix, 
                                                               output, weights, l1_penalty)
            weights_i_change = abs(old_weights_i - weights[feature_i][0])
            if weights_i_change > max_weights_change:
                max_weights_change = weights_i_change
    return(weights)

Using the following parameters, learn the weights on the sales dataset. 

In [22]:
simple_features = ["sqft_living", "bedrooms"]
my_output = ["price"]
initial_weights = np.zeros(3).reshape(-1,1)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [23]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [24]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTIONS***
* **What is the RSS of the learned model on the normalized dataset?**


In [25]:
RSS_norm = np.square(output - predict_output(normalized_simple_feature_matrix, weights)).sum()
print("RSS normalised dataset:",RSS_norm)

RSS normalised dataset: 1.63049247672e+15


* **Which features had weight zero at convergence?**

In [26]:
weights

array([[ 21624997.95951908],
       [ 63157247.20788956],
       [        0.        ]])

Features having weight 0 is **bedrooms**

# Evaluating LASSO fit with more features

### Load train and test sets.

In [27]:
train_data = pd.read_csv("./data/kc_house_train_data.csv", dtype=sales_dtype)
test_data = pd.read_csv("./data/kc_house_test_data.csv", dtype=sales_dtype)

Let us consider the following set of features.

In [28]:
all_features = ["bedrooms",
                "bathrooms",
                "sqft_living",
                "sqft_lot",
                "floors",
                "waterfront", 
                "view", 
                "condition", 
                "grade",
                "sqft_above",
                "sqft_basement",
                "yr_built", 
                "yr_renovated"]

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [29]:
(all_features_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_features_matrix, all_features_norms) = normalize_features(all_features_matrix) # normalize features

## l1_penalty=1e7
First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [30]:
initial_weights = np.zeros(all_features_matrix.shape[1]).reshape(-1,1)
l1_penalty = 1e7
tolerance = 1.0

In [31]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

**What features had non-zero weight in this case?**

In [32]:
weights1e7_index = all_features.copy()
weights1e7_index.insert(0,"intercept")
all_features_weights1e7 = pd.DataFrame(weights1e7, columns=["weights1e7"], index=weights1e7_index)
print(all_features_weights1e7)

                 weights1e7
intercept      2.442960e+07
bedrooms       0.000000e+00
bathrooms      0.000000e+00
sqft_living    4.838917e+07
sqft_lot       0.000000e+00
floors         0.000000e+00
waterfront     3.317511e+06
view           7.329962e+06
condition      0.000000e+00
grade          0.000000e+00
sqft_above     0.000000e+00
sqft_basement  0.000000e+00
yr_built       0.000000e+00
yr_renovated   0.000000e+00


In [33]:
print(all_features_weights1e7[all_features_weights1e7["weights1e7"] != 0])

               weights1e7
intercept    2.442960e+07
sqft_living  4.838917e+07
waterfront   3.317511e+06
view         7.329962e+06


## l1_penalty=1e8
Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [34]:
initial_weights = np.zeros(all_features_matrix.shape[1]).reshape(-1,1)
l1_penalty = 1e8
tolerance = 1.0

In [35]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

**What features had non-zero weight in this case?**

In [36]:
weights1e8_index = all_features.copy()
weights1e8_index.insert(0,"intercept")
all_features_weights1e8 = pd.DataFrame(weights1e8, columns=["weights1e8"], index=weights1e8_index)
print(all_features_weights1e8)

                 weights1e8
intercept      7.111463e+07
bedrooms       0.000000e+00
bathrooms      0.000000e+00
sqft_living    0.000000e+00
sqft_lot       0.000000e+00
floors         0.000000e+00
waterfront     0.000000e+00
view           0.000000e+00
condition      0.000000e+00
grade          0.000000e+00
sqft_above     0.000000e+00
sqft_basement  0.000000e+00
yr_built       0.000000e+00
yr_renovated   0.000000e+00


In [37]:
print(all_features_weights1e8[all_features_weights1e8["weights1e8"] != 0])

             weights1e8
intercept  7.111463e+07


## l1_penalty=1e4
Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [38]:
initial_weights = np.zeros(all_features_matrix.shape[1]).reshape(-1,1)
l1_penalty = 1e4
tolerance = 5e5

In [39]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_features_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

**What features had non-zero weight in this case?**

In [40]:
weights1e4_index = all_features.copy()
weights1e4_index.insert(0,"intercept")
all_features_weights1e4 = pd.DataFrame(weights1e4, columns=["weights1e8"], index=weights1e4_index)
print(all_features_weights1e4)

                 weights1e8
intercept      7.856474e+07
bedrooms      -2.209740e+07
bathrooms      1.279107e+07
sqft_living    9.380809e+07
sqft_lot      -2.013173e+06
floors        -4.219185e+06
waterfront     6.482843e+06
view           7.127409e+06
condition      5.001665e+06
grade          1.432752e+07
sqft_above    -1.577096e+07
sqft_basement -5.159591e+06
yr_built      -8.449534e+07
yr_renovated   2.824439e+06


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [41]:
weights1e4_normalised = weights1e4 / all_features_norms.reshape(-1,1)
weights1e7_normalised = weights1e7 / all_features_norms.reshape(-1,1)
weights1e8_normalised = weights1e8 / all_features_norms.reshape(-1,1)

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [42]:
weights1e7_normalised[3][0]

161.31745764611762

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [43]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, my_output)

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [44]:
RSS_weights1e4 = np.square(test_output - predict_output(test_feature_matrix, weights1e4_normalised)).sum()
print("RSS_weights1e4:",RSS_weights1e4)

RSS_weights1e4: 2.28459958971e+14


In [45]:
RSS_weights1e7 = np.square(test_output - predict_output(test_feature_matrix, weights1e7_normalised)).sum()
print("RSS_weights1e7:",RSS_weights1e7)

RSS_weights1e7: 2.7596207592e+14


In [46]:
RSS_weights1e8 = np.square(test_output - predict_output(test_feature_matrix, weights1e8_normalised)).sum()
print("RSS_weights1e8:",RSS_weights1e8)

RSS_weights1e8: 5.37166151497e+14


***QUIZ QUESTION***

**Which model performed best on the test data?**

RSS_weights1e4: 2.28459958971e+14